In [9]:
# import needed modules
import time, random

# define our functions
def inc(x):
    time.sleep(random.random())
    return x + 1

def dec(x):
    time.sleep(random.random())
    return x - 1

def add(x, y):
    time.sleep(random.random())
    return x + y

In [10]:
%%time

# a sequential example with no parallelization
results = []
for x in range(20):
    result = inc(x)
    result = dec(result)
    results.append(result)

print(results)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
CPU times: user 1.21 s, sys: 78.5 ms, total: 1.29 s
Wall time: 18.3 s


In [11]:
# import dask for parallel work
from dask.distributed import Client, LocalCluster

In [17]:
# Set up our cluster with default workers and threads
cluster = LocalCluster(processes=False)
cluster

/srv/conda/envs/notebook/lib/python3.11/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37437 instead
  warnings.warn(


LocalCluster(8cece661, 'inproc://192.168.54.53/2332/44', workers=1, threads=4, memory=1.85 GiB)

In [18]:
%%time
# Set up a client for work
client = cluster.get_client()

results = []
for x in range(20):
    result = client.submit(inc, x)
    result = client.submit(dec, result)
    results.append(result)

results = client.gather(results)
print(results)
client.close()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
CPU times: user 1.14 s, sys: 50.1 ms, total: 1.19 s
Wall time: 6.52 s


In [19]:
# When we are done we can close our dask cluster
cluster.close()

In [7]:
# Set up a new cluster with default 4 workers and 1 thread per worker
cluster = LocalCluster(n_workers=4, processes=False, threads_per_worker=1)
cluster

LocalCluster(41c55849, 'inproc://192.168.54.53/2332/10', workers=4, threads=4, memory=1.85 GiB)

In [8]:
%%time
# Set up a client for work
client = cluster.get_client()

results = []
for x in range(20):
    result = client.submit(inc, x)
    result = client.submit(dec, result)
    results.append(result)

results = client.gather(results)
print(results)
client.close()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
CPU times: user 831 ms, sys: 39.9 ms, total: 871 ms
Wall time: 5.58 s


In [15]:
# When we are done we can close our dask cluster
cluster.close()

In [16]:
cluster

Dashboard: /user/eeholmes/proxy/43165/status,Workers: 0
Total threads: 0,Total memory: 0 B
Status: closed,Using processes: False
Comm: inproc://192.168.54.53/2332/38,Workers: 0
Dashboard: /user/eeholmes/proxy/43165/status,Total threads: 0
Started: Just now,Total memory: 0 B
